In [ ]:
%run decision_tree.ipynb

import pandas as pd
import numpy as np

# Hàm lấy các mẫu dữ liệu ngẫu nhiên trong đó các phần tử có thể lặp lại (trùng nhau)
def bootstrap(X, y):  # X là DataFrame, y là Series
    n_sample = X.shape[0]
    _id = np.random.choice(n_sample, n_sample, replace=True)  # Chọn ngẫu nhiên với phép lặp lại
    return X.iloc[_id], y.iloc[_id]  # Dùng iloc vì đang chọn index ngẫu nhiên

# Lớp RandomForest
class RandomForest:
    def __init__(self, n_trees=5, max_depth=10, min_samples_split=2, n_features=None):
        self.n_trees = n_trees  # Số cây quyết định trong rừng
        self.max_depth = max_depth  # Độ sâu tối đa của từng cây
        self.min_samples_split = min_samples_split  # Số mẫu tối thiểu để tiếp tục chia node
        self.n_features = n_features  # Số lượng đặc trưng sử dụng trong mỗi cây
        self.trees = []  # Danh sách để lưu các cây

    def fit(self, X, y):  # X là DataFrame, y là Series
        self.trees = []  # Tạo danh sách mới chứa các cây
        for i in range(self.n_trees):
            # Khởi tạo cây quyết định cho mỗi lần lặp
            tree = DecisionTreeClass(
                min_samples_split=self.min_samples_split, 
                max_depth=self.max_depth, 
                n_features=self.n_features
            )
            X_sample, y_sample = bootstrap(X, y)  # Tạo mẫu bootstrap từ dữ liệu gốc
            tree.fit(X_sample, y_sample)  # Huấn luyện cây với mẫu mới
            self.trees.append(tree)  # Lưu cây vào danh sách

    def predict(self, X):  # Dự đoán cho toàn bộ tập dữ liệu
        # Lấy dự đoán từ từng cây trong rừng
        arr_pred = np.array([tree.predict(X) for tree in self.trees])  # Kết quả là mảng 2D
        final_pred = []
        for i in range(arr_pred.shape[1]):  # Duyệt qua từng mẫu dữ liệu
            sample_pred = arr_pred[:, i]  # Lấy tất cả dự đoán cho mẫu thứ i
            final_pred.append(most_value(pd.Series(sample_pred)))  # Bỏ phiếu đa số
        return np.array(final_pred)  # Trả về mảng dự đoán cuối cùng
